# Dataset Preprocessing

This notebook exemplifies the generation of pseudo LiDAR training data for one CARLA town.

## Setup

In [ ]:
import os
from pathlib import Path
import json

import numpy as np
from tqdm import tqdm

from pseudolidar.lidar_generator import PseudoLidarGenerator
from pseudolidar.pcl_generator import PointCloudGenerator

In [ ]:
DATASET_ROOT = Path('./data/')
TOWN = '05'
EXTRINSICS_PATH = Path('pseudolidar/extrinsics.json')
FOV = 100
IMAGE_DIM = (300, 400)

NUM_LIDAR_MEASUREMENTS = 250
MAX_DEPTH_FRONT = 0.05
MAX_DEPTH_SIDE = 0.03

In [ ]:
# Helper function to read camera extrinsics

def read_poses(path: os.PathLike):
    with open(path, mode='r') as f:
        poses = json.load(f)
    
    front = PseudoLidarGenerator.load_pose(poses['cam_front'])
    left = PseudoLidarGenerator.load_pose(poses['cam_left'])
    right = PseudoLidarGenerator.load_pose(poses['cam_right'])

    return [front, left, right]

In [ ]:
# Pseudo-LiDAR setup

poses = read_poses(EXTRINSICS_PATH)
pc_generator = PointCloudGenerator(FOV, IMAGE_DIM)
lidar_generator = PseudoLidarGenerator(poses)

## Pseudo-LiDAR generation

The following block generates the pseudo-LiDAR data for the selected town. Note that this takes about 1 hour per town.

In [ ]:
town_path = DATASET_ROOT / f'Town{TOWN}_short'
routes = [p for p in town_path.iterdir() if p.is_dir() and p.name.startswith('routes_')]
for route in tqdm(routes):
    fronts = sorted([p for p in route.joinpath('depth_front').iterdir() if p.is_file() and p.suffix == '.png'])
    lefts = sorted([p for p in route.joinpath('depth_left').iterdir() if p.is_file() and p.suffix == '.png'])
    rights = sorted([p for p in route.joinpath('depth_right').iterdir() if p.is_file() and p.suffix == '.png'])
    
    out_dir = route / 'pseudolidar'
    out_dir.mkdir(exist_ok=True)

    for front, left, right in zip(fronts, lefts, rights):
        pcs = [pc_generator.generate(front, max_depth=MAX_DEPTH_FRONT),
                pc_generator.generate(left, max_depth=MAX_DEPTH_SIDE),
                pc_generator.generate(right, max_depth=MAX_DEPTH_SIDE)
        ]

        pc = lidar_generator.generate(pcs)
        mesh = PseudoLidarGenerator.reconstruct_mesh(pc)
        points = PseudoLidarGenerator.raycast_mesh(mesh, n_measurements=NUM_LIDAR_MEASUREMENTS)
        
        np.save(out_dir / f'{front.stem}.npy', points)